In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json 

import glob
import tqdm

pd.set_option("max.columns", 131)

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df = df[df["y"].notnull()]
df.shape

(498, 16)

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [4]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo["title"] = df["watch-title"]

## 1. Limpeza de data

In [5]:
# criando um data frame, dividindo em colunas DIA-MÊS-ANO, utilizando expressão regular
clean_date = df["watch-time-text"].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
# adicionando 0 a esquerda, em dias abaixo de 10
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)

mapa_meses = {"jan": "Jan",
              "fev": "Feb", 
              "mar": "Mar",
              "abr": "Apr",
              "mai": "May",
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug",
              "set": "Sep",
              "out": "Oct",
              "nov": "Nov",
              "dez": "Dec"}
# Substituindo o mês para o formato inglês
clean_date[1] = clean_date[1].map(mapa_meses)
# Transformando tudo em uma linha (e não váris colunas com estava)
clean_date = clean_date.apply(lambda x: " ".join(x), axis = 1)
# jogando para o DataFrame criado anteriormente, em formato DATETIME
df_limpo["date"] = pd.to_datetime(clean_date, format= "%d %b %Y") 

## 2. Limpeza de views

In [6]:
# extraindo apenas os valores inteiros, retirando o "." do milhar
# atribuindo 0 para vídeos com "Nenhuma vizualização"
views = df["watch-view-count"].str.extract(r"(\d+\.?\d+)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo["views"] = views

## 3. Features

In [7]:
# criando um novo DataFrame com indice igual ao criando anteriormente
features = pd.DataFrame(index=df_limpo.index)
# atribuindo minhas labels a ele (se o vídeo é bom ou ruim)
y = df["y"].copy()
# Atribuição da Data atual (da coleta de dados), menos data de publicação =  Quantos dias o vídeo está no ar
# np.timedelta64 -> para fazer a diferença em dias
features["tempo_desde_pub"] = (pd.to_datetime("2019-12-3") - df_limpo["date"]) / np.timedelta64(1, "D")
# atribuição da quantidade de views
features["views"] = df_limpo["views"]
# conta para obter a média de views por dia
features["views_por_dia"] = features["views"] / features["tempo_desde_pub"]
# após análise, vimos que não é preciso essa coluna
features = features.drop(["tempo_desde_pub"], axis = 1)

In [8]:
mask_train = df_limpo["date"] < "2019-04-01"
mask_val = df_limpo["date"] >= "2019-04-01"
# Primeiro modelo apenas com duas features (views e views por dia) 
# para analisar o que elas indicam, seu efeito e suas previsões
# Divisão não exata, com mais ou menos 50% pra treino e 50% para teste
Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((228, 2), (270, 2), (228,), (270,))

Nessa parte, iremos obter os títulos dos vídeos já divididos (treino e validação) e temos que criar uma matriz de palavras com quantidade que elas se repetem, sendo cada coluna uma palavra e linhas os valores das quantidades de vezes que cada palavra aparece.
<br> Para realizar isso utilizamos o TfidVectorizer, que basicamente é uma formulinha que da peso as palavras. Palavras que aparecem bastante em um determinado exemplo, mas não aparece tanto no dataset inteiro <b>(aparece muito em um vídeo, porém pouco em relação a todos)</b>, terão valores maiores. Palavras muito comuns, terão valores menores.

In [9]:
# bow = bag of words (bolsa de palavras)

from sklearn.feature_extraction.text import TfidfVectorizer
# atribui títulos dos vídeos de treino
title_train = df_limpo[mask_train]['title']
# atribui títulos dos vídeos de validação
title_val = df_limpo[mask_val]['title']

#min_df = número mínimo que uma palavra aparece nos dados para se tornar uma coluna
# Vetorizador transforma os valores em uma matriz
title_vec = TfidfVectorizer(min_df=2)
# fit armazena palavras que o objeto viu, indicando que elas estão no documento
title_bow_train = title_vec.fit_transform(title_train)
# validação só usa o método transform. Pois se usar o fit iremos ensinar palavras para o vetorizador, que eu não saberia.
# ex: eu não sei o título de um vídeo do youtube que vai entrar amanhã.
title_bow_val = title_vec.transform(title_val)

Notamos que o modelo nos retorna uma matriz esparça, isto é, ele só armazena valores diferente de 0 pois se ele tivesse que armazenar <b> todos os elementos, seriam 44004 elementos</b>, porém ele só armazena 1277.

In [10]:
title_bow_train

<228x193 sparse matrix of type '<class 'numpy.float64'>'
	with 1277 stored elements in Compressed Sparse Row format>

In [11]:
title_bow_train.shape

(228, 193)

In [12]:
228*193

44004

Juntando variavéis númericas com variavéis de texto

In [13]:
Xtrain.head()

,views,views_por_dia
0,28028,61.464912
1,1131,2.960733
4,1228,3.336957
6,430097,903.565126
7,88592,308.682927


Entendendo hstack, vstack

In [14]:
# acrescentando mais uma classe aos nossos dados que irá passar pelo modelo.
from scipy.sparse import hstack, vstack
Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [15]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((228, 195), (270, 195))

In [16]:
# Criando um modelo de árvore de decisão aleatória
# n_estimartors = número de árvores que será criada
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=4)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=4, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [17]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

Utilizando métricas já vistas anteriormente.

In [18]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [19]:
average_precision_score(yval, p)

0.19055404076207902

In [20]:
roc_auc_score(yval, p)

0.5841945288753798

<b>Com mindf = 2, temos:</b>
<br>average_precision = 0.19
<br>roc = 0.58

<br><br><b>Com mindf = 1, temos: </b>
<br>average_precision = 0.17
<br>roc = 0.60

<br><br><b>Com mindf = 3, temos: </b>
<br>average_precision = 0.19
<br>roc = 0.53

Mantemos mindf = 2, pois com mindf = 2 a chance de overfitting é menor.

# 5. Active Learning

In [21]:
df_unlabeled = pd.read_csv("raw_data_with_labels.csv", index_col=0)
# selecionando todas as linhas que o y = null
# dropna how = all -> tira todas as linhas que todos os campos estejam NaN
df_unlabeled = df_unlabeled[df_unlabeled["y"].isnull()].dropna(how="all")

In [23]:
df_unlabeled.head(1)

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
501,Kaggle Mercari Price Suggestion Challenge (1 p...,NaN,2.167 visualizações,Publicado em 2 de nov. de 2018,Educação,Kaggle Mercari Price Suggestion Challenge (1 p...,ML Trainings\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarreg...,2.167 visualizações\n\n\n\n\n\n\n\n61\n\nGosto...,https://i.ytimg.com/vi/QFR0IHbzA30/maxresdefau...,1280.0,720.0,Pawel Jankiewicz and Konstantin Lopuhin share ...,1280.0,720.0,price suggestion,/channel/UCeq6ZIlvC9SVsfhfKnSvM9w


In [24]:
df_limpo_u = pd.DataFrame(index=df_unlabeled.index)
df_limpo_u["title"] = df_unlabeled["watch-title"]

In [25]:
clean_date = df_unlabeled["watch-time-text"].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)

mapa_meses = {"jan": "Jan",
              "fev": "Feb", 
              "mar": "Mar",
              "abr": "Apr",
              "mai": "May",
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug",
              "set": "Sep",
              "out": "Oct",
              "nov": "Nov",
              "dez": "Dec"}
clean_date[1] = clean_date[1].map(mapa_meses)
clean_date = clean_date.apply(lambda x: " ".join(x), axis = 1)
df_limpo_u["date"] = pd.to_datetime(clean_date, format= "%d %b %Y") 

In [26]:
df_limpo_u.head()

,title,date
501,Kaggle Mercari Price Suggestion Challenge (1 p...,2018-11-02
502,OpenAI Gym and Python for Q-learning - Reinfor...,2018-10-14
503,"Dashboarding with Notebooks, Day 1: What infor...",2018-12-17
504,How To Get US- American Company H1 Visa To Get...,2019-11-23
505,Platform Overview - Machine Learning,2019-05-21


In [27]:
views = df_unlabeled["watch-view-count"].str.extract(r"(\d+\.?\d+)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo_u["views"] = views

In [28]:
features_u = pd.DataFrame(index=df_limpo_u.index)
features_u["tempo_desde_pub"] = (pd.to_datetime("2019-12-3") - df_limpo_u["date"]) / np.timedelta64(1, "D")
features_u["views"] = df_limpo_u["views"]
features_u["views_por_dia"] = features_u["views"] / features_u["tempo_desde_pub"]
features_u = features_u.drop(["tempo_desde_pub"], axis = 1)

In [29]:
features_u.head()

,views,views_por_dia
501,2167,5.472222
502,20378,49.103614
503,10435,29.729345
504,0,0.000000
505,4298,21.928571


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
title_u = df_limpo_u["title"]
title_bow_u = title_vec.transform(title_u)

In [35]:
title_bow_u

<674x193 sparse matrix of type '<class 'numpy.float64'>'
	with 3079 stored elements in Compressed Sparse Row format>

In [36]:
Xu_wtitle = hstack([features_u, title_bow_u])

In [37]:
Xu_wtitle

<674x195 sparse matrix of type '<class 'numpy.float64'>'
	with 4349 stored elements in COOrdinate format>

In [39]:
pu = mdl.predict_proba(Xu_wtitle)[:, 1]

In [40]:
# acrescentando uma nova coluna p ("probabilidade"), para os exemplos
df_unlabeled["p"] = pu

In [42]:
df_unlabeled.head(1)

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
501,Kaggle Mercari Price Suggestion Challenge (1 p...,NaN,2.167 visualizações,Publicado em 2 de nov. de 2018,Educação,Kaggle Mercari Price Suggestion Challenge (1 p...,ML Trainings\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarreg...,2.167 visualizações\n\n\n\n\n\n\n\n61\n\nGosto...,https://i.ytimg.com/vi/QFR0IHbzA30/maxresdefau...,1280.0,720.0,Pawel Jankiewicz and Konstantin Lopuhin share ...,1280.0,720.0,price suggestion,/channel/UCeq6ZIlvC9SVsfhfKnSvM9w,0.093


Tentativas até encontrar aproximadamente 70 exemplos.

In [46]:
mask_u = (df_unlabeled["p"] >= 0.26) & (df_unlabeled["p"] <= 1.)
mask_u.sum()

69

In [47]:
mask_u

501     False
502     False
503     False
504     False
505      True
        ...  
1179    False
1180    False
1181    False
1182    False
1183    False
Name: p, Length: 674, dtype: bool

In [44]:
df_unlabeled[mask_u]

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
505,Platform Overview - Machine Learning,NaN,4.298 visualizações,Publicado em 21 de mai. de 2019,Ciência e tecnologia,Platform Overview - Machine Learning,Google Cloud Platform\n\n\n\n\n\n\n\n\n\n\n\n\...,4.298 visualizações\n\n\n\n\n\n\n\n141\n\nGost...,https://i.ytimg.com/vi/QR_LQQ-vvko/maxresdefau...,1280.0,720.0,"In this short GCP Essentials video, see how GC...",1280.0,720.0,Alexis Moussine Pouchkine,/channel/UCJS9pqu9BzkAMNTmzNMNhvg,0.502
507,Kaggle Meetup: Ship Detection Challenge,NaN,504 visualizações,Publicado em 30 de nov. de 2018,Ciência e tecnologia,Kaggle Meetup: Ship Detection Challenge,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,504 visualizações\n\n\n\n\n\n\n\n9\n\nGostou d...,https://i.ytimg.com/vi/QXEy4rdLsDw/maxresdefau...,1280.0,720.0,Video from the 2018-11-29 meetup. Kaggle page:...,1280.0,720.0,learn data science,/channel/UCJhW_16uxALr0X4olEW2p5A,0.455
535,Identifying art through machine learning at MoMA,NaN,9.476 visualizações,Publicado em 7 de mar. de 2018,Entretenimento,#MachineLearning #Musuem #Art\n\n\n\n Ident...,Google Arts & Culture\n\n\n\n\n\n\n\n\n\n\n\n\...,9.476 visualizações\n\n\n\n\n\n\n\n49\n\nGosto...,https://i.ytimg.com/vi/SLBqVOnn9Mo/maxresdefau...,1280.0,720.0,"The Art Recognizer, built in collaboration wit...",1280.0,720.0,museum of modern art,/channel/UCGn7dlcAmH44GqycKa_3ssA,0.510
557,Deep Learning - Computerphile,NaN,131.286 visualizações,Publicado em 6 de jun. de 2018,Educação,Deep Learning - Computerphile,Computerphile\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCar...,131.286 visualizações\n\n\n\n\n\n\n\n3.700\n\n...,https://i.ytimg.com/vi/TJlAxW-2nmI/maxresdefau...,1280.0,720.0,Deep Learning with Convolutional Neural Networ...,1280.0,720.0,Neural Network,/channel/UC9-y-6csu5WGm29I7JiwpnA,0.453
705,Kaggle Meetup: CANSSI NCSC Ferry Delays,NaN,124 visualizações,Publicado em 3 de out. de 2019,Ciência e tecnologia,Kaggle Meetup: CANSSI NCSC Ferry Delays,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,124 visualizações\n\n\n\n\n\n\n\n2\n\nGostou d...,https://i.ytimg.com/vi/aOGnEHywIBc/maxresdefau...,1280.0,720.0,Presenter: Matt Tourond Kaggle page: https://w...,1280.0,720.0,machine learning,/channel/UCJhW_16uxALr0X4olEW2p5A,0.464
718,Kaggle Meetup: Human Protein Atlas,NaN,226 visualizações,Publicado em 8 de fev. de 2019,Ciência e tecnologia,Kaggle Meetup: Human Protein Atlas,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,226 visualizações\n\n\n\n\n\n\n\n5\n\nGostou d...,https://i.ytimg.com/vi/bAgjqdQ0VE8/maxresdefau...,1280.0,720.0,Kaggle page: https://www.kaggle.com/c/human-pr...,1280.0,720.0,kaggle,/channel/UCJhW_16uxALr0X4olEW2p5A,0.487
744,Data Science Specialization at IFMR-GSB,NaN,207 visualizações,Publicado em 27 de dez. de 2018,Educação,Data Science Specialization at IFMR-GSB,IFMR GSB KREA University\n\n\n\n\n\n\n\n\n\n\n...,207 visualizações\n\n\n\n\n\n\n\n3\n\nGostou d...,https://i.ytimg.com/vi/crUfuo9GXIc/maxresdefau...,1280.0,720.0,,1280.0,720.0,NaN,/channel/UCAjwb2trX_m_6hobx02MV9g,0.469
904,Kaggle Meetup: AdTracking Fraud Detection,NaN,372 visualizações,Publicado em 1 de jun. de 2018,Ciência e tecnologia,Kaggle Meetup: AdTracking Fraud Detection,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,372 visualizações\n\n\n\n\n\n\n\n11\n\nGostou ...,https://i.ytimg.com/vi/kzkQTPeZPQ4/maxresdefau...,1280.0,720.0,https://www.meetup.com/LearnDataScience/events...,1280.0,720.0,NaN,/channel/UCJhW_16uxALr0X4olEW2p5A,0.521
923,Shri Debasish Sen at 7th International Data Sc...,NaN,5 visualizações,Publicado em 23 de nov. de 2019,Ciência e tecnologia,Shri Debasish Sen at 7th International Data Sc...,Data Science Foundation\n\n\n\n\n\n\n\n\n\n\n\...,5 visualizações\n\n\n\n\nGostou deste vídeo?\n...,https://i.ytimg.com/vi/m0NTnHGSfkA/maxresdefau

In [48]:
dificies = df_unlabeled[mask_u]

Pegando 31 amostras que não são confusas.
<br> ~mascara -> exemplos contrários

In [50]:
aleatorios = df_limpo_u[~mask_u].sample(31)

In [54]:
pd.concat([dificies, aleatorios]).to_csv("active_label1.csv")

In [55]:
dificies.head()

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
505,Platform Overview - Machine Learning,NaN,4.298 visualizações,Publicado em 21 de mai. de 2019,Ciência e tecnologia,Platform Overview - Machine Learning,Google Cloud Platform\n\n\n\n\n\n\n\n\n\n\n\n\...,4.298 visualizações\n\n\n\n\n\n\n\n141\n\nGost...,https://i.ytimg.com/vi/QR_LQQ-vvko/maxresdefau...,1280.0,720.0,"In this short GCP Essentials video, see how GC...",1280.0,720.0,Alexis Moussine Pouchkine,/channel/UCJS9pqu9BzkAMNTmzNMNhvg,0.502
507,Kaggle Meetup: Ship Detection Challenge,NaN,504 visualizações,Publicado em 30 de nov. de 2018,Ciência e tecnologia,Kaggle Meetup: Ship Detection Challenge,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,504 visualizações\n\n\n\n\n\n\n\n9\n\nGostou d...,https://i.ytimg.com/vi/QXEy4rdLsDw/maxresdefau...,1280.0,720.0,Video from the 2018-11-29 meetup. Kaggle page:...,1280.0,720.0,learn data science,/channel/UCJhW_16uxALr0X4olEW2p5A,0.455
519,flowlightさん『TalkingData AdTracking Fraud Detec...,NaN,1.707 visualizações,Publicado em 12 de mai. de 2018,Pessoas e blogs,flowlightさん『TalkingData AdTracking Fraud Detec...,Takami Sato\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarrega...,1.707 visualizações\n\n\n\n\n\n\n\n17\n\nGosto...,https://i.ytimg.com/vi/RKXUEJVJJ-o/maxresdefau...,1280.0,720.0,資料 https://www.slideshare.net/TakanoriHayashi3...,1280.0,720.0,kaggle,/channel/UCiECS_auJLNpFsvjTi1WuxQ,0.283
527,Reinforcement Learning with TensorFlow and Uni...,NaN,545 visualizações,Publicado em 22 de nov. de 2019,Ciência e tecnologia,Reinforcement Learning with TensorFlow and Uni...,Google Developers\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,545 visualizações\n\n\n\n\n\n\n\n13\n\nGostou ...,https://i.ytimg.com/vi/S-MbpQiwfls/maxresdefau...,1280.0,720.0,"Dan Goncharov, Head of 42 Robotics GDG Fremont...",1280.0,720.0,Dan Goncharov,/channel/UC_x5XG1OV2P6uZZ5FSM9Ttw,0.261
535,Identifying art through machine learning at MoMA,NaN,9.476 visualizações,Publicado em 7 de mar. de 2018,Entretenimento,#MachineLearning #Musuem #Art\n\n\n\n Ident...,Google Arts & Culture\n\n\n\n\n\n\n\n\n\n\n\n\...,9.476 visualizações\n\n\n\n\n\n\n\n49\n\nGosto...,https://i.ytimg.com/vi/SLBqVOnn9Mo/maxresdefau...,1280.0,720.0,"The Art Recognizer, built in collaboration wit...",1280.0,720.0,museum of modern art,/channel/UCGn7dlcAmH44GqycKa_3ssA,0.510


In [56]:
aleatorios.head()

,title,date,views
571,Kaggle Reading Group: Dissecting contextual wo...,2019-03-13,1016
1053,We talk about machine learning,2019-11-25,0
763,Time Series Analysis in Python | Time Series F...,2018-05-31,151104
964,Big Data y Data Science,2019-11-22,0
666,From Deep Learning of Disentangled Representat...,2018-02-08,37182
